In [4]:
import onnxruntime as rt
from sklearn.metrics import accuracy_score
from scipy.stats import wilcoxon
import pandas as pd
import numpy as np
from testing import *

In [5]:
# First we load the models and the data 
data = pd.read_csv('data/investigation_train_large_checked.csv')
y = data['checked']
X = data.drop(['checked', 'Ja', 'Nee'], axis=1)
X = X.astype(np.float32)

# model1 = rt.InferenceSession("model/model_1.onnx")
model2 = rt.InferenceSession("C:\\Users\\waded\\Downloads\\group20\\model_2\\model_2\\model_2.onnx")


# Mutation testing

In [ ]:
mutationTestResults = mutationTest(model1,model2,X,y,10)
compareAccuracies(mutationTestResults)


Run 1/10


AttributeError: 'InferenceSession' object has no attribute 'predict'

In [ ]:
# Wilcoxon Test
stat, p_value = wilcoxon(mutationTestResults['model1'], mutationTestResults['model1_mutated'])
print("\nWilcoxon Test Results for Model 1:")
print(f"Statistic: {stat:.4f}, P-value: {p_value:.4f}")

if p_value < 0.05:
    print("Significant difference in performance between original and mutated models.")
else:
    print("No significant difference in performance between original and mutated models.")

 # Wilcoxon Test
stat, p_value = wilcoxon(mutationTestResults['model2'], mutationTestResults['model2_mutated'])
print("\nWilcoxon Test Results for Model 2:")
print(f"Statistic: {stat:.4f}, P-value: {p_value:.4f}")

if p_value < 0.05:
    print("Significant difference in performance between original and mutated models.")
else:
    print("No significant difference in performance between original and mutated models.")

# Differentiation Testing


In [ ]:
features_to_modify = ['contacten_soort_document__uitgaand_' , 'afspraak_aantal_woorden' ,'persoon_leeftijd_bij_onderzoek','relatie_kind_leeftijd_verschil_ouder_eerste_kind','persoonlijke_eigenschappen_spreektaal']

metrics = differentiationTesting(model1,model2,X,y,0.2,features_to_modify,10)
print_average_metrics(metrics)

# Equivalence partitioning 

In [ ]:
print("Top 5 worst performing features for model 1: ")
model1_results = calculateEPHighestDifference(data,model1)
print(model1_results)

print("Top 5 worst performing features for model 2: ")
model2_results = calculateEPHighestDifference(data,model2)
print(model2_results)

In [ ]:
gender_partitions = [
    {"name": "Man", "condition": lambda df: df['persoon_geslacht_vrouw'] < 1},
    {"name": "Woman", "condition": lambda df: (df['persoon_geslacht_vrouw'] >= 1)}
    
]

EPResults = equivalencePartitioning(model1,model2,X,y,gender_partitions,1)

plot_EP_results(EPResults,'accuracy')
plot_EP_results(EPResults,'tp')
plot_EP_results(EPResults,'tn')
plot_EP_results(EPResults,'fp')
plot_EP_results(EPResults,'fn')